In [ ]:
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join
from natsort import os_sorted
from PIL import Image

path = r'\\fatherserverdw\Q\research\images\CLUE\3D study\he\1.25x\divided\TA1\resize'
filenames = listdir(path)
filenames = os_sorted(filenames)
impath0 = join(path, filenames[0]) # first image (reference)
impath1 = join(path, filenames[1]) # second image

# Open the image files.
img1_color = cv2.imread(impath1)  # Image to be aligned.
img2_color = cv2.imread(impath0)    # Reference image.

# Convert to grayscale.
img1 = cv2.cvtColor(img1_color, cv2.COLOR_BGR2GRAY)
img2 = cv2.cvtColor(img2_color, cv2.COLOR_BGR2GRAY)
height, width = img2.shape

# Create ORB detector with 5000 features.
orb_detector = cv2.ORB_create(5000)

# Find keypoints and descriptors.
# The first arg is the image, second arg is the mask
#  (which is not required in this case).
kp1, d1 = orb_detector.detectAndCompute(img1, None)
kp2, d2 = orb_detector.detectAndCompute(img2, None)

# Match features between the two images.
# We create a Brute Force matcher with
# Hamming distance as measurement mode.
matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck = True)

# Match the two sets of descriptors.
matches = matcher.match(d1, d2)

# Sort matches on the basis of their Hamming distance.
matches.sort(key = lambda x: x.distance)

# Take the top 90 % matches forward.
matches = matches[:int(len(matches)*0.9)]
no_of_matches = len(matches)

# Define empty matrices of shape no_of_matches * 2.
p1 = np.zeros((no_of_matches, 2))
p2 = np.zeros((no_of_matches, 2))

for i in range(len(matches)):
  p1[i, :] = kp1[matches[i].queryIdx].pt
  p2[i, :] = kp2[matches[i].trainIdx].pt

# Find the homography matrix.
homography, mask = cv2.findHomography(p1, p2, cv2.RANSAC)

# Use this matrix to transform the
# colored image wrt the reference image.
transformed_img = cv2.warpPerspective(img1_color,
                    homography, (width, height))

# Save the output.
# cv2.imwrite('output.jpg', transformed_img)

In [ ]:
ax0.imshow(img1_color)
ax0.set_title("Image to be aligned")
ax0.set_axis_off()

ax1.imshow(img2_color)
ax1.set_title("Reference")
ax1.set_axis_off()

ax2.imshow(transformed_img)
ax2.set_title("Transformed")
ax2.set_axis_off()

fig.tight_layout()